In [1]:
import pandas as pd
from sklearn import svm

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.metrics import accuracy_score
import pickle
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print('success')
'hello world'

success


'hello world'

In [2]:
'''Load data set'''
df1 = pd.read_csv("weightxy_data1.csv")
print(df1.shape)
print(df1.iloc[:,:-1].shape)
'hello world'

(2313, 2969)
(2313, 2968)


'hello world'

In [22]:
# X_train, X_test, Y_train, Y_test = train_test_split(df1.iloc[:,:-1], df1['class_'], test_size = 0.3, random_state =42)
X, Y = df1.iloc[:,:-1], df1['class_']
# svm_clf = svm.SVC(kernel = 'rbf')
svm_clf = svm.LinearSVC()
print('success')
print(X_train.shape)
print(Y_train.shape)
'hello world'

success
(2313, 2968)
(2313,)


'hello world'

In [40]:
'''Cross validation'''
scores = cross_validate(svm_clf, X_train, Y_train, cv = 10, scoring = 'accuracy')

print(scores.keys())
print('success')
'hello world'

dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success


'hello world'

In [35]:
print(scores)

{'fit_time': array([ 0.36422086,  0.43131614,  0.41930652,  0.43277931,  0.42857289,
        0.42609239,  0.40029001,  0.43706083,  0.42860079,  0.418612  ]), 'score_time': array([ 0.        ,  0.        ,  0.00200438,  0.00694156,  0.        ,
        0.00300908,  0.        ,  0.        ,  0.01571608,  0.        ]), 'test_score': array([ 0.64397033,  0.67050172,  0.70153802,  0.6782678 ,  0.65733938,
        0.68299247,  0.70575973,  0.69857398,  0.71782521,  0.70349063]), 'train_score': array([ 0.98493501,  0.98543145,  0.98414877,  0.98414412,  0.98345718,
        0.98667012,  0.98702581,  0.98336107,  0.98422796,  0.98335597])}


In [33]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaBoost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2)) #, n_estimators=2, learning_rate=1)
scores = cross_validate(adaBoost_clf, X_train, Y_train, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)


dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success
{'fit_time': array([ 5.5008359 ,  5.07896113,  5.28814936,  5.3848424 ,  5.37402415,
        5.06019902,  4.89597917,  5.25338435,  5.48487449,  5.2946074 ]), 'score_time': array([ 0.04982114,  0.04562497,  0.02989864,  0.03485727,  0.04191017,
        0.04948449,  0.04350209,  0.04970217,  0.04002285,  0.04534721]), 'test_score': array([ 0.59482759,  0.65086207,  0.61637931,  0.59913793,  0.62068966,
        0.65086207,  0.67532468,  0.61304348,  0.5826087 ,  0.65217391]), 'train_score': array([ 0.77126382,  0.76501682,  0.76838059,  0.7534839 ,  0.75060067,
        0.76549736,  0.76176753,  0.76572252,  0.75084013,  0.76428229])}


In [41]:
# '''calculate the precision recall and stuff'''

# t_precision = precision_score(Y_test, t_pred, average='weighted')
# t_recall = recall_score(Y_test, t_pred, average='weighted')
# t_f1_score = f1_score(Y_test, t_pred, average='weighted')

# print('*** Scores for SVM: ***')
# target_names = ['0', '1', '-1']
# print('Pr, Re scores wrt each class')
# print(classification_report(Y_test, t_pred, target_names=target_names))

# print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))
# print('\n\nPrecision Score is: {:.4%}' .format(t_precision))
# print('\n\nRecall Score is: {:.4%}' .format(t_recall))
# print('\n\nF1-Score is: {:.4%}' .format(t_f1_score))
